# Functions to Save Aggregated Calls Data to CSV

In [158]:
import requests
import csv
import pandas as pd
import time
def fetch_aggregated_calls(ticker, start_date, end_date, interval_minutes):
    """
    Fetches aggregated volumes and calculates price volatility for the given stock ticker and date range.

    Args:
    - ticker (str): The stock ticker symbol.
    - start_date (str): The start date in the format 'YYYY-MM-DD'.
    - end_date (str): The end date in the format 'YYYY-MM-DD'.
    - interval_minutes (int): The interval in minutes for the aggregation.

    Returns:
    - pd.DataFrame: A DataFrame containing aggregated data with timestamps as index.
    """
    #stock volatility
    stock_volatility = {
        'AMZN': 0.24725763030035341,
        'MSFT': 0.4896242798552114,
        'TSLA': 0.6813622018551234,
        'GOOG': 0.21620848553976152,
        'NVDA': 1.0125333149293283,
        'META': 0.6043551329552291,
        'AAPL': 0.23929615724381625,
        'QQQ': 0.393723454063604,
        'SPY': 0.3544695781802127,
        'AMD': 0.3218514686395759
    }

    # API query parameters
    api_key = "RLH5skiQVl24hXBE2IyPT3JBDmorI4r0"


    # Construct initial API query URL
    stock_data = {}
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{interval_minutes}/minute/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"
    params = {
            'apiKey': api_key
        }
    start_trading_time = "09:30"
    end_trading_time = "10:00"
    
    while url:
        # Make the API request
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Load JSON data from the response
            data = response.json()
            results = data["results"]

            # Iterate through results and process data
            for result in results:
                timestamp = result["t"]
                num_trades = result["n"]
                volume = result["v"]
                open_price = result["o"]
                close = result["c"]
                change = close - open_price
                timestamp_et = pd.to_datetime(timestamp, unit='ms', utc=True).tz_convert('America/New_York')
                timestamp_str = timestamp_et.strftime("%Y-%m-%d %H:%M:%S")
                trading_time = timestamp_et.strftime("%H:%M")
                if change >= stock_volatility[ticker] and start_trading_time <= trading_time <= end_trading_time:
                    stock_data[timestamp_str] = {"change": change, "num_trades": num_trades, "volume": volume}

            # Update URL for next page if available
            url = data.get('next_url')

            # Delay to comply with rate limit (5 requests per minute)
            time.sleep(12)  # Sleep for 12 seconds between each request

        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            break
    
    # Convert dictionary to DataFrame
    df = pd.DataFrame(stock_data).T

    # Save DataFrame to CSV
    df.to_csv(f"{ticker}_aggregated_calls.csv")

    return df

In [159]:
tickers = ['AMZN', 'MSFT', 'TSLA', 'GOOG', 'NVDA', 'META', 'AAPL', 'AMD']
start_date = '2023-01-01'
end_date = '2024-02-18'
interval_minutes = 5
for ticker in tickers: 
    volumes = fetch_aggregated_calls(ticker, start_date, end_date, interval_minutes)

KeyboardInterrupt: 

# Functions to Save Aggregated Puts Data to CSV

In [160]:
import requests
import csv
import pandas as pd
import time
def fetch_aggregated_puts(ticker, start_date, end_date, interval_minutes):
    """
    Fetches aggregated volumes and calculates price volatility for the given stock ticker and date range.

    Args:
    - ticker (str): The stock ticker symbol.
    - start_date (str): The start date in the format 'YYYY-MM-DD'.
    - end_date (str): The end date in the format 'YYYY-MM-DD'.
    - interval_minutes (int): The interval in minutes for the aggregation.

    Returns:
    - pd.DataFrame: A DataFrame containing aggregated data with timestamps as index.
    """
    #stock volatility
    stock_volatility = {
        'AMZN': 0.24725763030035341,
        'MSFT': 0.4896242798552114,
        'TSLA': 0.6813622018551234,
        'GOOG': 0.21620848553976152,
        'NVDA': 1.0125333149293283,
        'META': 0.6043551329552291,
        'AAPL': 0.23929615724381625,
        'QQQ': 0.393723454063604,
        'SPY': 0.3544695781802127,
        'AMD': 0.3218514686395759
    }

    # API query parameters
    api_key = "RLH5skiQVl24hXBE2IyPT3JBDmorI4r0"


    # Construct initial API query URL
    stock_data = {}
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{interval_minutes}/minute/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"
    params = {
            'apiKey': api_key
        }
    start_trading_time = "09:30"
    end_trading_time = "10:00"
    
    while url:
        # Make the API request
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Load JSON data from the response
            data = response.json()
            results = data["results"]

            # Iterate through results and process data
            for result in results:
                timestamp = result["t"]
                num_trades = result["n"]
                volume = result["v"]
                open_price = result["o"]
                close = result["c"]
                change = close - open_price
                timestamp_et = pd.to_datetime(timestamp, unit='ms', utc=True).tz_convert('America/New_York')
                timestamp_str = timestamp_et.strftime("%Y-%m-%d %H:%M:%S")
                trading_time = timestamp_et.strftime("%H:%M")
                if abs(change) >= stock_volatility[ticker] and start_trading_time <= trading_time <= end_trading_time and change < 0:
                    stock_data[timestamp_str] = {"change": change, "num_trades": num_trades, "volume": volume}

            # Update URL for next page if available
            url = data.get('next_url')

            # Delay to comply with rate limit (5 requests per minute)
            time.sleep(12)  # Sleep for 12 seconds between each request

        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            break
    
    # Convert dictionary to DataFrame
    df = pd.DataFrame(stock_data).T

    # Save DataFrame to CSV
    df.to_csv(f"{ticker}_aggregated_puts.csv")

    return df

In [161]:
tickers = ['AMZN', 'MSFT', 'TSLA', 'GOOG', 'NVDA', 'META', 'AAPL', 'AMD']
start_date = '2023-01-01'
end_date = '2024-02-18'
interval_minutes = 5
for ticker in tickers: 
    volumes = fetch_aggregated_puts(ticker, start_date, end_date, interval_minutes)

# Functions to Save Aggregated RSI Data to CSV

In [162]:
import requests
import pandas as pd
import time
import pytz

def fetch_rsi_data_within_time_range(stock_ticker, start_timestamp, end_timestamp):
    """
    Fetches Relative Strength Index (RSI) data for the given stock ticker within a specified time range.

    Args:
    - stock_ticker (str): The ticker symbol for which to get RSI data.
    - start_timestamp (str): The start timestamp in the format 'YYYY-MM-DD'.
    - end_timestamp (str): The end timestamp in the format 'YYYY-MM-DD'.

    Returns:
    - dict: A dictionary containing RSI data with timestamps as keys.
    """
    # API endpoint
    api_key = "RLH5skiQVl24hXBE2IyPT3JBDmorI4r0"
    url = f"https://api.polygon.io/v1/indicators/rsi/{stock_ticker}"

    # Initialize dictionary to store RSI data
    rsi_data = {}
    start_trading_time = "09:30"
    end_trading_time = "10:00"


    # Continue fetching data until all data is fetched
    while url:
        # Construct query parameters
        params = {
            'timestamp.gt': start_timestamp,
            'timestamp.lt': end_timestamp,
            'limit': 5000,
            'timespan': "minute", 
            'apiKey': api_key,
            'window': 9
        }

        # Make the API request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            results = data["results"]["values"]
            for result in results:
                timestamp = result["timestamp"]
                value = result["value"]
                timestamp_et = pd.to_datetime(timestamp, unit='ms', utc=True).tz_convert('America/New_York')
                timestamp_str = timestamp_et.strftime("%Y-%m-%d %H:%M:%S")
                trading_time = timestamp_et.strftime("%H:%M")
                if start_trading_time <= trading_time <= end_trading_time and timestamp_et.minute % 5 == 0:
                    timestamp_str = timestamp_et.strftime("%Y-%m-%d %H:%M:%S")
                    rsi_data[timestamp_str] = {"rsi": value}
            url = data.get('next_url')
            time.sleep(12)  # Sleep for 12 seconds between each request
        
    # Convert dictionary to DataFrame
    df = pd.DataFrame(rsi_data).T

    # Save DataFrame to CSV
    df.to_csv(f"{stock_ticker}_rsi.csv")

    return df

In [163]:
tickers = ['AMZN', 'MSFT', 'TSLA', 'GOOG', 'NVDA', 'META', 'AAPL', 'AMD']
start_timestamp = '2023-01-01T00:00:00Z'  
end_timestamp = '2024-02-18T16:00:00Z'    
for ticker in tickers: 
    volumes = fetch_rsi_data_within_time_range(ticker, start_date, end_date)